# mix produttivo

Un’azienda utilizza tre reparti (1, 2 e 3) per produrre due composti (A e B).

Ogni kg di A e B richiede un dato numero di ore di lavorazione in ciascun reparto e ogni reparto ha una data capacità produttiva, ossia è disponibile per un certo numero di ore al giorno (vedi schema).

![img](./img.png)

![tab](./tab.png)

Considerato che ogni composto ha un dato prezzo di vendita al kg (ultima colonna della tabella), qual è la produzione di A e B che, nel rispetto delle capacità produttive dei reparti, massimizza il profitto dell’azienda?


## formulazione generale del problema di mix produttivo

Un’azienda produce n articoli $P_1$,...,$P_n$ utilizzando un impianto con $m$ reparti $R_1$,...,$R_m$.

Ogni unità dell’articolo $P_i$ assicura all’azienda un profitto netto di $p_i$ € e richiede $a_{ij}$ ore di lavorazione nel reparto $R_j$.

Qual è il mix produttivo giornaliero più redditizio per l’azienda, cioè quali sono le quantità di ogni articolo che l’azienda deve produrre per massimizzare il profitto rispettando le capacità produttive dei reparti?


## modello parametrico

![param1](./param1.png)

![param2](./param2.png)